In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import re
import itertools
from typing import Union, List, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import time
import difflib
import numpy as np
from sklearn.neighbors import NearestNeighbors
import Levenshtein
import requests
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import json 
from transformers import BertTokenizer, BertForSequenceClassification
import nltk
from nltk.corpus import wordnet 

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/englishwords.json') as f:
    data = json.load(f)

words = data.keys()
words = list(words)

In [ ]:
class TrieNode_dict:
    def __init__(self):
        self.children = {}
        self.is_word = False
        self.description = None

class Trie_dict:
    def __init__(self):
        self.root = TrieNode_dict()
    
    def insert(self, word, description = None):
        current = self.root
        for char in word:
            if char not in current.children:
                current.children[char] = TrieNode_dict()
            current = current.children[char]
        current.is_word = True
        current.description = description
    
    def search(self, word):
        current = self.root
        for char in word:
            if char not in current.children:
                return None
            current = current.children[char]
        if current.is_word:
            return current.description
        return None

    def insert_list(self, lst):
        for word in lst:
            self.insert(word) 
    
    def fuzzy_search(self, word, cutoff=0.6):
        results = difflib.get_close_matches(word, self.words(), n=10, cutoff=cutoff)
        return {result: (self.search(result), difflib.SequenceMatcher(None, word, result).ratio()) for result in results}

    def fuzzy_search_knn(self, word, k=5, cutoff=0.4):
        words = np.array(self.words())
        words_len = np.array([len(w) for w in words])
        word_len = len(word)
        distances = np.abs(words_len - word_len)
        knn = NearestNeighbors(n_neighbors=k, metric='manhattan')
        knn.fit(distances.reshape(-1, 1))
        _, indices = knn.kneighbors(np.array([word_len]).reshape(-1, 1))
        results = [words[index] for index in indices[0]]
        ratio = [difflib.SequenceMatcher(None, word, result).ratio() for result in results]
        return {result: (self.search(result), ratio[i]) for i, result in enumerate(results) if ratio[i] >= cutoff}
        
        
    def words(self):
        words = []
        def dfs(node, word):
            if node.is_word:
                words.append(word)
            for char in node.children:
                dfs(node.children[char], word + char)
        dfs(self.root, "")
        return words


def extract_description(trie, key):
    return trie.search(key)

trie_dict = Trie_dict()

trie_dict.insert_list(words)

In [ ]:
import wordcloud

nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

synsets = wordnet.synsets('')
for synset in synsets:
    print(synset.definition())

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from nltk.corpus import wordnet

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Define a function to fine-tune the BERT model
def fine_tune(word, definition):
    if definition is None:
        label = torch.tensor([[0.0]])
    else:
        label = torch.tensor([[1.0]])

    print(f"Fine-tuning the model with '{word}': '{definition}'")
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-6)
    model.train()
    for epoch in range(20):
        inputs = tokenizer(word, definition, return_tensors='pt', padding=True, truncation=True)
        optimizer.zero_grad()
        outputs = model(**inputs, labels=torch.tensor([[1.0]]))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Check if the definition of the word is in WordNet
def get_definition(word):
    synsets = wordnet.synsets(word)
    if synsets:
        return synsets[0].definition()
    else:
        return "None"

def predict_definition(word, description):
    inputs = tokenizer(word, description, return_tensors='pt', padding=True, truncation=True)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = outputs.logits.sigmoid()
    words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    prediction_indices = torch.where(probabilities > 0.1)[1]
    prediction = [words[i] for i in prediction_indices]
    #definition = ' '.join(prediction) or "Sorry, I could not predict the definition of this word."
    return definition

In [ ]:
def dl_definition():
  # Ask the user for a word
  word = input("Enter a word: ")

  # Check if the definition of the word is in the BERT model's knowledge base
  definition = get_definition(word)
  if definition:
      print(f"The definition of '{word}' is: {definition}")
      trie_dict.insert(word, definition)
  else:
      print(f"Sorry, I don't know the definition of '{word}'.")
      definition = input(f"Please enter the definition of '{word}': ")
      fine_tune(word, definition)
      print(f"Thanks, I've learned the definition of '{word}'.")
      predicted_definition = predict_definition(word, definition)
      new_string = "".join(predicted_definition)
      new_string = new_string.replace('#', '')
      new_string = new_string.replace('[CLS]', '')
      new_string = new_string.replace('[SEP]', ' ')
      print(f"The predicted definition of '{word}' is: {new_string}")
      print(type(predicted_definition))
      trie_dict.insert(word, new_string)

In [ ]:
search = input('What would you like to search?')
search_result = list(trie_dict.fuzzy_search(search))
print(search_result)

In [ ]:
def navigate_trie(trie_dict):  
  search = input('What would you like to search?')
  search_result = list(trie_dict.fuzzy_search(search))
  
  if search_result == "":
    print('Not in our list')
    print('Define it and att it to the model!!')
    dl_definition()
  else:
    print(search_result)
    print('Is the desired term in the list?')
    isit = input('[yes/no]')
    if isit == 'yes':
        print("Please select an item from the list:")
        for i, item in enumerate(search_result):
            print(f"{i+1}. {item}")
        while True:
            try:
                user_choice = int(input("Enter the number corresponding to your choice: "))
                if 1 <= user_choice <= len(search_result):
                    break
                else:
                    print(f"Invalid choice. Please enter a number between 1 and {len(search_result)}")
            except ValueError:
                print("Invalid input. Please enter a number.")

        selected_item = search_result[user_choice - 1]
        print(f"You selected: {selected_item}")
        url = f"https://api.dictionaryapi.dev/api/v2/entries/en/{selected_item}"
        response = requests.get(url)


        try:

          definition = response.json()[0]["meanings"][0]["definitions"][0]["definition"]
          custom_def = trie_dict.search(selected_item)

          if custom_def == None:

            print(f'No previous definition has been found, however {selected_item} is commonly referred to as:\n ')
            print(f"Definition : {definition}")
            print('Is this definition satisfactory? If it is, type "Y", otherwise type anything else')
            choice_description = input('Choice: ')
            if choice_description == 'Y':
              trie_dict.insert(selected_item, definition)
            else:
              definition = input('Then insert your defintion: ')
              trie_dict.insert(selected_item, definition)
          
          else:
            print(custom_def)
            print('Is this definition satisfactory? If it is, type "Y", otherwise type anything else')
            choice_description = input('Choice: ')
            if choice_description == 'Y':
              trie_dict.insert(selected_item, custom_def)
            else:
              custom_def = input('Then insert your defintion: ')
              trie_dict.insert(selected_item, custom_def)
              
        except KeyError:
            print('Definition not found, would you like to add it? Type "Y" if so')
            choice_description = input('Choice: ')
            if choice_description == 'Y':
              custom_def = input('Then insert your defintion: ')
              trie_dict.insert(selected_item, custom_def)      
            else:
              print('Have a good day!')
    else:
      print("Would you like to input it in the system, along with a definition?")
      input_choice  = input('[yes/no]');
      if input_choice == 'yes':
        word = input('Input word')
        definition = input('Input associated definition')
        trie_dict.insert(word, definition)

In [ ]:
class my_dictionary(dict):
 
  # __init__ function
  def __init__(self):
    self = dict()
 
  # Function to add key:value
  def add(self, key, value):
    self[key] = value
 
 
# Main Function
dict_obj = my_dictionary()
glossary_terms_ds = pd.DataFrame(columns = [['Term', 'Definition']])
df = pd.DataFrame(list(dict_obj.items()), columns=['Terms', 'Definitions'])
df.to_csv('wordnet_english_dictionary.csv')

# Trying another BERT to train it on the English dictionary again but failing miserably

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd
# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Tokenize the input and output tokens
input_tokens = [tokenizer.encode(input_word, add_special_tokens=False) for input_word in df['Terms']]
output_tokens = [tokenizer.encode(output_word, add_special_tokens=False) for output_word in df['Definitions']]

# Convert the tokens into PyTorch tensors
input_tensors = [torch.tensor(input_token) for input_token in input_tokens]
output_tensors = [torch.tensor(output_token) for output_token in output_tokens]
